In [6]:
# 파이토치

import torch
import torch.nn as nn

class Bottleneck(nn.Module):

    def __init__(self, in_ch, growth_rate):
        super(Bottleneck, self).__init__()
        layers = []
        layers += [nn.BatchNorm2d(in_ch),
                   nn.ReLU(True),
                   nn.Conv2d(in_ch, growth_rate*4, kernel_size=1, bias=False),
                   nn.BatchNorm2d(growth_rate*4),
                   nn.ReLU(True),
                   nn.Conv2d(growth_rate*4, growth_rate, kernel_size=3, stride=1, padding=1, bias=False)]
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        out = self.layers(x)
        out = torch.cat((x, out), dim=1)
        return out

class DenseBlock(nn.Module):

    def __init__(self, n_layers, n_ch, growth_rate):
        super(DenseBlock, self).__init__()

        for i in range(n_layers):
            setattr(self, "Dense_layer_{}".format(i), Bottleneck(n_ch + i * growth_rate, growth_rate))

        self.n_layers = n_layers

    def forward(self, x):
        for i in range(self.n_layers):
            x = getattr(self, "Dense_layer_{}".format(i))(x)
        return x

class Transition_layer(nn.Module):

    def __init__(self, in_ch):
        super(Transition_layer, self).__init__()
        num_ch = int(in_ch*0.5)
        layers = []
        layers += [nn.BatchNorm2d(in_ch),
                   nn.ReLU(True),
                   nn.Conv2d(in_ch, num_ch, kernel_size=3, padding=1, bias=False),
                   nn.AvgPool2d(kernel_size=2, stride=2)]
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

class DenseNet(nn.Module):

    def __init__(self):
        super(DenseNet, self).__init__()
        layers = []
        layers += [nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
                   nn.BatchNorm2d(64),
                   nn.ReLU(True),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
                   DenseBlock(6, 64, 32),
                   Transition_layer(256),
                   DenseBlock(12, 128, 32),
                   Transition_layer(512),
                   DenseBlock(24, 256, 32),
                   Transition_layer(1024),
                   DenseBlock(16, 512, 32),
                   nn.AdaptiveAvgPool2d((1, 1)),
                   View(-1),
                   nn.Linear(1024, 10)]
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        x = self.layers(x)
        return x

class View(nn.Module):

    def __init__(self, *shape):
        super(View, self).__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(x.shape[0], *self.shape)

if __name__ == '__main__':
    from torchsummary import summary
    model = DenseNet()
    summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
       BatchNorm2d-5             [-1, 64, 8, 8]             128
              ReLU-6             [-1, 64, 8, 8]               0
            Conv2d-7            [-1, 128, 8, 8]           8,192
       BatchNorm2d-8            [-1, 128, 8, 8]             256
              ReLU-9            [-1, 128, 8, 8]               0
           Conv2d-10             [-1, 32, 8, 8]          36,864
       Bottleneck-11             [-1, 96, 8, 8]               0
      BatchNorm2d-12             [-1, 96, 8, 8]             192
             ReLU-13             [-1, 96, 8, 8]               0
           Conv2d-14            [-1, 12

          Conv2d-125            [-1, 128, 4, 4]          57,344
     BatchNorm2d-126            [-1, 128, 4, 4]             256
            ReLU-127            [-1, 128, 4, 4]               0
          Conv2d-128             [-1, 32, 4, 4]          36,864
      Bottleneck-129            [-1, 480, 4, 4]               0
     BatchNorm2d-130            [-1, 480, 4, 4]             960
            ReLU-131            [-1, 480, 4, 4]               0
          Conv2d-132            [-1, 128, 4, 4]          61,440
     BatchNorm2d-133            [-1, 128, 4, 4]             256
            ReLU-134            [-1, 128, 4, 4]               0
          Conv2d-135             [-1, 32, 4, 4]          36,864
      Bottleneck-136            [-1, 512, 4, 4]               0
      DenseBlock-137            [-1, 512, 4, 4]               0
     BatchNorm2d-138            [-1, 512, 4, 4]           1,024
            ReLU-139            [-1, 512, 4, 4]               0
          Conv2d-140            [-1, 256

          Conv2d-253             [-1, 32, 2, 2]          36,864
      Bottleneck-254            [-1, 768, 2, 2]               0
     BatchNorm2d-255            [-1, 768, 2, 2]           1,536
            ReLU-256            [-1, 768, 2, 2]               0
          Conv2d-257            [-1, 128, 2, 2]          98,304
     BatchNorm2d-258            [-1, 128, 2, 2]             256
            ReLU-259            [-1, 128, 2, 2]               0
          Conv2d-260             [-1, 32, 2, 2]          36,864
      Bottleneck-261            [-1, 800, 2, 2]               0
     BatchNorm2d-262            [-1, 800, 2, 2]           1,600
            ReLU-263            [-1, 800, 2, 2]               0
          Conv2d-264            [-1, 128, 2, 2]         102,400
     BatchNorm2d-265            [-1, 128, 2, 2]             256
            ReLU-266            [-1, 128, 2, 2]               0
          Conv2d-267             [-1, 32, 2, 2]          36,864
      Bottleneck-268            [-1, 832

            ReLU-381            [-1, 800, 1, 1]               0
          Conv2d-382            [-1, 128, 1, 1]         102,400
     BatchNorm2d-383            [-1, 128, 1, 1]             256
            ReLU-384            [-1, 128, 1, 1]               0
          Conv2d-385             [-1, 32, 1, 1]          36,864
      Bottleneck-386            [-1, 832, 1, 1]               0
     BatchNorm2d-387            [-1, 832, 1, 1]           1,664
            ReLU-388            [-1, 832, 1, 1]               0
          Conv2d-389            [-1, 128, 1, 1]         106,496
     BatchNorm2d-390            [-1, 128, 1, 1]             256
            ReLU-391            [-1, 128, 1, 1]               0
          Conv2d-392             [-1, 32, 1, 1]          36,864
      Bottleneck-393            [-1, 864, 1, 1]               0
     BatchNorm2d-394            [-1, 864, 1, 1]           1,728
            ReLU-395            [-1, 864, 1, 1]               0
          Conv2d-396            [-1, 128

In [1]:
# 케라스

import keras

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Activation, BatchNormalization, concatenate, AveragePooling2D, GlobalAveragePooling2D
from keras.models import Model

growth_rate = 32
theta = 0.5

def DenseBlock(x, num):
    layer_list = [x]
    for i in range(num): # layer 수가 늘어날수록 layer_list에 있는 x에 대해 새로운 x가 concatenate 된다
        x1 = BatchNormalization()(x)
        x1 = Activation('relu')(x1)
        x1 = Conv2D(growth_rate*4, kernel_size=1, padding='same', use_bias=False)(x1)
        x1 = BatchNormalization()(x1)
        x1 = Activation('relu')(x1)
        x1 = Conv2D(growth_rate, kernel_size=3, padding='same', use_bias=False)(x1)
        layer_list.append(x1)
        x = concatenate(layer_list, axis=-1) # channel에 대해 layer들을 concatenate
    return x


def Transition_layer(x, name):
    num_ch = int(x.get_shape().as_list()[-1] * theta) # theta 비율만큼 channel 수를 줄임
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_ch, kernel_size=3, padding='same', use_bias=False)(x)
    x = AveragePooling2D(pool_size=2, strides=2, padding='same', name=name)(x)
    
    return x


inputs = Input(shape=(224, 224, 3))
x = Conv2D(64, kernel_size=7, strides=2, padding='same', use_bias=False)(inputs) # (None,112, 112, 64)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = MaxPooling2D(pool_size=3, strides=2, padding='same')(x) # (None,56, 56, 64)

x = DenseBlock(x, 6) # (None,56, 56, 256)
x = Transition_layer(x, 'transition_layer_1') # (None,28, 28, 128)
x = DenseBlock(x, 12) # (None,28, 28, 512)
x = Transition_layer(x, 'transition_layer_2') # (None,14, 14, 256)
x = DenseBlock(x, 24) # (None,14, 14, 1024)
x = Transition_layer(x, 'transition_layer_3') # (None,7, 7, 512)
x = DenseBlock(x, 16) # (None,7, 7, 1024)
x = GlobalAveragePooling2D()(x) # (None,1024)
outputs = Dense(10, activation='softmax')(x) # (None, 10)

model = Model(inputs=inputs, outputs=outputs)

model.summary()

Using TensorFlow backend.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9408        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

activation_81 (Activation)      (None, 14, 14, 128)  0           batch_normalization_81[0][0]     
__________________________________________________________________________________________________
conv2d_81 (Conv2D)              (None, 14, 14, 32)   36864       activation_81[0][0]              
__________________________________________________________________________________________________
concatenate_39 (Concatenate)    (None, 14, 14, 928)  0           transition_layer_2[0][0]         
                                                                 conv2d_41[0][0]                  
                                                                 conv2d_43[0][0]                  
                                                                 conv2d_45[0][0]                  
                                                                 conv2d_47[0][0]                  
                                                                 conv2d_49[0][0]                  
          

In [ ]:
# 모델 훈련 - 훈련은 CIFAR10. 훈련시키려면 input shape을 (32, 32, 3)으로 해야됨.

from keras.datasets import cifar10
from keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

from keras.optimizers import SGD

optimizer = SGD(lr=0.1, decay=0.001, momentum=0.9, nesterov=True)
model.compile(optimizer, loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, batch_size=256, epochs=90, validation_split=0.2)